# Download name.basics.tsv and Read

In [1]:
import pandas as pd
actors = pd.read_csv('/Users/apillai2/Downloads/name.basics.tsv', sep='\t', header=0)

In [2]:
actors.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0072308,tt0050419,tt0053137"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0117057,tt0038355,tt0071877"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0057345,tt0056404,tt0054452,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0072562,tt0080455,tt0078723,tt0077975"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050976,tt0060827,tt0083922,tt0050986"


In [3]:
len(actors)

11359935

# Apply Filters

In [4]:
actors = actors[(actors.birthYear > '1960') & (actors.primaryProfession.str.contains('act'))]
actors['titleCount'] = actors['knownForTitles'].str.count(',')+1
actors = actors[(actors.titleCount > 3)]
print(len(actors))
actors = actors[0:2500]
actors = actors.reset_index(drop=True)
print(len(actors))
actors.head()

771701
2500


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,titleCount
0,nm0000084,Gong Li,1965,\N,actress,"tt0101640,tt0430357,tt0397535,tt0473444",4
1,nm0000093,Brad Pitt,1963,\N,"actor,producer,soundtrack","tt0114746,tt1210166,tt0356910,tt2935510",4
2,nm0000096,Gillian Anderson,1968,\N,"actress,producer,soundtrack","tt0106179,tt2294189,tt0455590,tt0442632",4
3,nm0000097,Pamela Anderson,1967,\N,"actress,producer,director","tt0115624,tt0267913,tt0306047,tt0426592",4
4,nm0000098,Jennifer Aniston,1969,\N,"actress,producer,soundtrack","tt1723121,tt0279113,tt0108778,tt3442006",4


# Check Samples

In [5]:
actors[actors.primaryName.str.contains('Brad Pitt')]

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,titleCount
1,nm0000093,Brad Pitt,1963,\N,"actor,producer,soundtrack","tt0114746,tt1210166,tt0356910,tt2935510",4


In [6]:
actors[actors.primaryName.str.contains('Chris Nolan')]

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,titleCount


# Get Unique Titles for Actors

In [7]:
allTitles = ['tt0118571']
for i in actors['knownForTitles'].tolist():
    allTitles=allTitles+i.split(',')

uniqTitles = list(set(allTitles))
len(uniqTitles)

7273

# Scrape Related Fuctions

In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
import re 
import urllib
import time

In [9]:
#create a webdriver object and set options for headless browsing
options = Options()
options.headless = True
driver = webdriver.Chrome('./chromedriver',options=options)
#driver = webdriver.Chrome("/usr/local/bin/chromedriver",options=options)

In [10]:
#uses webdriver object to execute javascript code and get dynamically loaded webcontent
def get_js_soup(url,driver):
    driver.get(url)
    res_html = driver.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(res_html,'html.parser') #beautiful soup object to be used for parsing html content
    return soup

#tidies extracted text 
def process_bio(bio):
    bio = bio.encode('ascii',errors='ignore').decode('utf-8')       #removes non-ascii characters
    bio = re.sub('\s+',' ',bio)       #repalces repeated whitespace characters with single space
    return bio

''' More tidying
Sometimes the text extracted HTML webpage may contain javascript code and some style elements. 
This function removes script and style tags from HTML so that extracted text does not contain them.
'''
def remove_script(soup):
    for script in soup(["script", "style"]):
        script.decompose()
    return soup


#Checks if bio_url is a valid faculty homepage
def is_valid_homepage(bio_url,dir_url):
    if bio_url.endswith('.pdf'): #we're not parsing pdfs
        return False
    try:
        #sometimes the homepage url points to the same page as the faculty profile page
        #which should be treated differently from an actual homepage
        ret_url = urllib.request.urlopen(bio_url).geturl() 
    except:
        return False       #unable to access bio_url
    urls = [re.sub('((https?://)|(www.))','',url) for url in [ret_url,dir_url]] #removes url scheme (https,http or www) 
    return not(urls[0]== urls[1])

def scrape_movie_page(fac_url,driver):
    soup = get_js_soup(fac_url,driver)
    homepage_found = False
    bio_url = fac_url
    #profile_sec = soup.find('section',class_='main-content')
    profile_sec = soup.find('section',class_='article listo')
    if profile_sec == None:
        bio = ''
        print("Skipping bio : ",bio_url)
    else:
        bio = process_bio(profile_sec.get_text(separator=' '))
    #print("bio is ",bio_url)
    return bio_url,bio

In [11]:
def write_lst(lst,file_):
    lst = lst.replace(' Plot Showing all 0 items Jump to: ','')
    lst = lst.replace(' Plot Showing all 1 items Jump to: Summaries (1) ','')
    lst = lst.replace(' Plot Showing all 2 items Jump to: Summaries (2) ','')
    lst = lst.replace(' Plot Showing all 3 items Jump to: Summaries (3) ','')
    lst = lst.replace(' Plot Showing all ','').replace(' items Jump to: Summaries (','')
    lst = lst.replace(') Synopsis (','').replace(' Synopsis','').replace(' Summaries','')
    #lst = lst[6:]
    lst = lst.replace('Edit ','')
    #lst = lst.replace(' Summaries ','\n\nSummaries\n')
    #lst = lst.replace(' Synopsis ','\n\nSynopsis\n')

    with open(file_,'w') as f:
        f.write(lst)

# Scrape Movie plot summary and store in "movieFile" dir 

In [12]:
#Scrape homepages of all urls
tot_urls = len(uniqTitles)
for i,l in enumerate(uniqTitles):
    link="https://www.imdb.com/title/"+l+"/plotsummary"
    print ('-'*20,'Scraping movie url {}/{}'.format(i+1,tot_urls),'-'*20)
    print(link)
    bio_urls, bios = [],[]
    bio_url,bio = scrape_movie_page(link,driver)
    if bio.strip()!= '' and bio_url.strip()!='':  
        bio_file = 'movieFile/'+l+'.txt'
        write_lst(bio,bio_file)
driver.close()

-------------------- Scraping movie url 1/7273 --------------------
https://www.imdb.com/title/tt0452594/plotsummary
-------------------- Scraping movie url 2/7273 --------------------
https://www.imdb.com/title/tt0440229/plotsummary
-------------------- Scraping movie url 3/7273 --------------------
https://www.imdb.com/title/tt0236195/plotsummary
-------------------- Scraping movie url 4/7273 --------------------
https://www.imdb.com/title/tt1488606/plotsummary
-------------------- Scraping movie url 5/7273 --------------------
https://www.imdb.com/title/tt0170508/plotsummary
-------------------- Scraping movie url 6/7273 --------------------
https://www.imdb.com/title/tt0094750/plotsummary
-------------------- Scraping movie url 7/7273 --------------------
https://www.imdb.com/title/tt5944526/plotsummary
-------------------- Scraping movie url 8/7273 --------------------
https://www.imdb.com/title/tt0117102/plotsummary
-------------------- Scraping movie url 9/7273 -----------------

-------------------- Scraping movie url 71/7273 --------------------
https://www.imdb.com/title/tt0060315/plotsummary
-------------------- Scraping movie url 72/7273 --------------------
https://www.imdb.com/title/tt4122692/plotsummary
-------------------- Scraping movie url 73/7273 --------------------
https://www.imdb.com/title/tt0082958/plotsummary
-------------------- Scraping movie url 74/7273 --------------------
https://www.imdb.com/title/tt0095016/plotsummary
-------------------- Scraping movie url 75/7273 --------------------
https://www.imdb.com/title/tt0099180/plotsummary
-------------------- Scraping movie url 76/7273 --------------------
https://www.imdb.com/title/tt0293549/plotsummary
-------------------- Scraping movie url 77/7273 --------------------
https://www.imdb.com/title/tt0412556/plotsummary
-------------------- Scraping movie url 78/7273 --------------------
https://www.imdb.com/title/tt0076941/plotsummary
-------------------- Scraping movie url 79/7273 --------

-------------------- Scraping movie url 141/7273 --------------------
https://www.imdb.com/title/tt0430051/plotsummary
-------------------- Scraping movie url 142/7273 --------------------
https://www.imdb.com/title/tt1235522/plotsummary
-------------------- Scraping movie url 143/7273 --------------------
https://www.imdb.com/title/tt1056471/plotsummary
-------------------- Scraping movie url 144/7273 --------------------
https://www.imdb.com/title/tt0180885/plotsummary
-------------------- Scraping movie url 145/7273 --------------------
https://www.imdb.com/title/tt0158671/plotsummary
-------------------- Scraping movie url 146/7273 --------------------
https://www.imdb.com/title/tt2077823/plotsummary
-------------------- Scraping movie url 147/7273 --------------------
https://www.imdb.com/title/tt2305051/plotsummary
-------------------- Scraping movie url 148/7273 --------------------
https://www.imdb.com/title/tt0314331/plotsummary
-------------------- Scraping movie url 149/7273

-------------------- Scraping movie url 210/7273 --------------------
https://www.imdb.com/title/tt1759761/plotsummary
-------------------- Scraping movie url 211/7273 --------------------
https://www.imdb.com/title/tt0245712/plotsummary
-------------------- Scraping movie url 212/7273 --------------------
https://www.imdb.com/title/tt0356249/plotsummary
-------------------- Scraping movie url 213/7273 --------------------
https://www.imdb.com/title/tt8106534/plotsummary
-------------------- Scraping movie url 214/7273 --------------------
https://www.imdb.com/title/tt0131334/plotsummary
-------------------- Scraping movie url 215/7273 --------------------
https://www.imdb.com/title/tt0330099/plotsummary
-------------------- Scraping movie url 216/7273 --------------------
https://www.imdb.com/title/tt5363178/plotsummary
-------------------- Scraping movie url 217/7273 --------------------
https://www.imdb.com/title/tt0451833/plotsummary
-------------------- Scraping movie url 218/7273

-------------------- Scraping movie url 279/7273 --------------------
https://www.imdb.com/title/tt0134269/plotsummary
-------------------- Scraping movie url 280/7273 --------------------
https://www.imdb.com/title/tt4329242/plotsummary
-------------------- Scraping movie url 281/7273 --------------------
https://www.imdb.com/title/tt0119859/plotsummary
-------------------- Scraping movie url 282/7273 --------------------
https://www.imdb.com/title/tt1527069/plotsummary
-------------------- Scraping movie url 283/7273 --------------------
https://www.imdb.com/title/tt0115396/plotsummary
-------------------- Scraping movie url 284/7273 --------------------
https://www.imdb.com/title/tt0094510/plotsummary
-------------------- Scraping movie url 285/7273 --------------------
https://www.imdb.com/title/tt2260324/plotsummary
-------------------- Scraping movie url 286/7273 --------------------
https://www.imdb.com/title/tt0109936/plotsummary
-------------------- Scraping movie url 287/7273

...

-------------------- Scraping movie url 6972/7273 --------------------
https://www.imdb.com/title/tt1937154/plotsummary
-------------------- Scraping movie url 6973/7273 --------------------
https://www.imdb.com/title/tt0120082/plotsummary
-------------------- Scraping movie url 6974/7273 --------------------
https://www.imdb.com/title/tt0313597/plotsummary
-------------------- Scraping movie url 6975/7273 --------------------
https://www.imdb.com/title/tt0362716/plotsummary
-------------------- Scraping movie url 6976/7273 --------------------
https://www.imdb.com/title/tt0179250/plotsummary
-------------------- Scraping movie url 6977/7273 --------------------
https://www.imdb.com/title/tt8861834/plotsummary
-------------------- Scraping movie url 6978/7273 --------------------
https://www.imdb.com/title/tt0806910/plotsummary
-------------------- Scraping movie url 6979/7273 --------------------
https://www.imdb.com/title/tt0090563/plotsummary
-------------------- Scraping movie url 

-------------------- Scraping movie url 7041/7273 --------------------
https://www.imdb.com/title/tt0089575/plotsummary
-------------------- Scraping movie url 7042/7273 --------------------
https://www.imdb.com/title/tt0234681/plotsummary
-------------------- Scraping movie url 7043/7273 --------------------
https://www.imdb.com/title/tt1369571/plotsummary
-------------------- Scraping movie url 7044/7273 --------------------
https://www.imdb.com/title/tt0189112/plotsummary
-------------------- Scraping movie url 7045/7273 --------------------
https://www.imdb.com/title/tt0098382/plotsummary
-------------------- Scraping movie url 7046/7273 --------------------
https://www.imdb.com/title/tt1515209/plotsummary
-------------------- Scraping movie url 7047/7273 --------------------
https://www.imdb.com/title/tt0450340/plotsummary
-------------------- Scraping movie url 7048/7273 --------------------
https://www.imdb.com/title/tt0050005/plotsummary
-------------------- Scraping movie url 

-------------------- Scraping movie url 7110/7273 --------------------
https://www.imdb.com/title/tt0275847/plotsummary
-------------------- Scraping movie url 7111/7273 --------------------
https://www.imdb.com/title/tt1895587/plotsummary
-------------------- Scraping movie url 7112/7273 --------------------
https://www.imdb.com/title/tt0214272/plotsummary
-------------------- Scraping movie url 7113/7273 --------------------
https://www.imdb.com/title/tt0134314/plotsummary
-------------------- Scraping movie url 7114/7273 --------------------
https://www.imdb.com/title/tt0434183/plotsummary
-------------------- Scraping movie url 7115/7273 --------------------
https://www.imdb.com/title/tt1441109/plotsummary
-------------------- Scraping movie url 7116/7273 --------------------
https://www.imdb.com/title/tt1073668/plotsummary
-------------------- Scraping movie url 7117/7273 --------------------
https://www.imdb.com/title/tt14095402/plotsummary
-------------------- Scraping movie url

-------------------- Scraping movie url 7179/7273 --------------------
https://www.imdb.com/title/tt0175526/plotsummary
-------------------- Scraping movie url 7180/7273 --------------------
https://www.imdb.com/title/tt8620070/plotsummary
-------------------- Scraping movie url 7181/7273 --------------------
https://www.imdb.com/title/tt1007028/plotsummary
-------------------- Scraping movie url 7182/7273 --------------------
https://www.imdb.com/title/tt0056693/plotsummary
-------------------- Scraping movie url 7183/7273 --------------------
https://www.imdb.com/title/tt2675978/plotsummary
-------------------- Scraping movie url 7184/7273 --------------------
https://www.imdb.com/title/tt0126906/plotsummary
-------------------- Scraping movie url 7185/7273 --------------------
https://www.imdb.com/title/tt0114745/plotsummary
-------------------- Scraping movie url 7186/7273 --------------------
https://www.imdb.com/title/tt0186589/plotsummary
-------------------- Scraping movie url 

-------------------- Scraping movie url 7248/7273 --------------------
https://www.imdb.com/title/tt0426883/plotsummary
-------------------- Scraping movie url 7249/7273 --------------------
https://www.imdb.com/title/tt2125453/plotsummary
-------------------- Scraping movie url 7250/7273 --------------------
https://www.imdb.com/title/tt2306299/plotsummary
-------------------- Scraping movie url 7251/7273 --------------------
https://www.imdb.com/title/tt2443340/plotsummary
-------------------- Scraping movie url 7252/7273 --------------------
https://www.imdb.com/title/tt0204679/plotsummary
-------------------- Scraping movie url 7253/7273 --------------------
https://www.imdb.com/title/tt0188937/plotsummary
-------------------- Scraping movie url 7254/7273 --------------------
https://www.imdb.com/title/tt0086143/plotsummary
-------------------- Scraping movie url 7255/7273 --------------------
https://www.imdb.com/title/tt0261392/plotsummary
-------------------- Scraping movie url 

# Concatenate movies for an actor and store as a single document

In [13]:
for i in range(len(actors)):
    fl_1 = actors['nconst'][i]
    #dataAct=actors['primaryName'][i]+':\n'
    dataAct=actors['nconst'][i]+':\n'
    for fl_2 in actors['knownForTitles'][i].split(','):
        fl_2 = 'movieFile/'+fl_2+'.txt'
        with open(fl_2, 'r') as file:
            data = file.read().replace('\n \n', '')
            dataAct = dataAct+'\n'+data
    act_file = 'actorFile/'+fl_1+'.txt'
    write_lst(dataAct,act_file) 

In [14]:
actors.to_csv('actors.csv',header=True,index=False)
actors.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,titleCount
0,nm0000084,Gong Li,1965,\N,actress,"tt0101640,tt0430357,tt0397535,tt0473444",4
1,nm0000093,Brad Pitt,1963,\N,"actor,producer,soundtrack","tt0114746,tt1210166,tt0356910,tt2935510",4
2,nm0000096,Gillian Anderson,1968,\N,"actress,producer,soundtrack","tt0106179,tt2294189,tt0455590,tt0442632",4
3,nm0000097,Pamela Anderson,1967,\N,"actress,producer,director","tt0115624,tt0267913,tt0306047,tt0426592",4
4,nm0000098,Jennifer Aniston,1969,\N,"actress,producer,soundtrack","tt1723121,tt0279113,tt0108778,tt3442006",4
